# Dissolution of Gypsum ($CaSO_4.2H_2O$)


In [1]:
import numpy as np
from IPython.display import Math
import scipy.optimize as scipyo

# Define constants for use in equations
Gf_Ca = -553.6 #kJ/mol
Gf_SO4 = -744.6
Gf_H2O = -237
logKeqGyps = -4.58  # {CaSO4.2H2O} = {Ca+2} + {SO4-2} + 2{H2O}
logKeqFluorite = -10.57

R = 8.3145E-03 #kJ/molK
T = 298.15 #K


In [2]:
# extra constants
mmGyps = 172.14 # molecular mass g/mol
Vol = 1 # liter


## Question 3 How much gypsum dissolves if you add gypsum to a solution of 0.05 mol/L of NaF?

In [3]:
# Required constants
Gf_Na = -261.9
Gf_F = -278.8
Gf_NaF = -546.3
mmNaF = 23+19



The assumption is that NaF is highly soluble and that we can assume that it will complete dissolve. 

If the amount of NaF that can dissolve is larger that the amount added, the assumption that NaF is highly soluble is valid and we can assume that all NaF will be in ionic form. Therefore our solution will now have the following solutes: $Ca^{2+}, Na^+, SO_4^{2-}$ and $F^-$ and the solid phase in our system will consist of $CaSO_4.2H_2O_{(s)}$ and $CaF_{2(s)}$.

If we want to solve this problem we have to find an equation base on the dissolution of gypsum and the precipitation of fluorite. To do this we need to write down the mass balance equations. Similar to the description in Appelo & Postma, the solution goes in two steps:
#1 We start in point A where no Ca+2 is in solution from where we move to a situation where solution is in equilibrium with Fluorite. We call this point B.
#2 We continue to dissolve gypsum, until the solution is in equilibrium with both Fluorite and Gypsum we call this Point C.

## From point A to point B
From the initial amount of F- in solution we estimate the concentration of Ca+2 and SO4-2 in point B using the solubility product of Fluorite.

In [4]:
# Concentration of F- in point A
F_A = 0.0005 #mol/l

Kfluorite = 10**logKeqFluorite
Kgyps = 10**logKeqGyps

Ca_B = Kfluorite / F_A**2
SO4_B = Ca_B
F_B = F_A

print(str(F_A),str(Ca_B))

0.0005 0.00010766139215707656


## From point B to point C
**Please note** that in this example we use a high initial concentration of $[F^-]$.

Moving from point B to point C, Ca+2 and SO4-2 keep on dissolving until the solubility product of Gypsum is reached, however simultaneously Fluorite will precipitate, removing Ca+2 from solution.

We start from condition (B) where no fluorite has precipitated (F_B = F_A) and Ca+2 has dissolved just until the solubility product of FLuorite. If we assume that the further amount of Ca+2 precipitating from B to C is x and F- precipitating is y, then we can write the following equations.

$$[Ca^{2+}]_C = [Ca^{2+}]_B + x - y$$
$$[F^-]_C = [F^-]_B - 2y$$
$$[SO_4^{2-}]_C = [SO_4^{2-}]_B + x$$
where we have 5 unknowns (the three concentrations in the final condition (B), x and y). Because we also know the equilibrium equations for the dissolution of gypsum and fluorite we have 5 equations with 5 unknowns which we can solve. However these equations are non-linear and we require non-linear solvers to do it.

A first approach can be done by making the assumption that y << x. In this case, this is not likely because the initial amount of Ca+2 at point B is so much smaller than the amount of F- present. We cannot make this assumption and we need to solve the equations for the two variables.

We first use the Fluorite to solve for Ca+2
which substituted in the solubility product of gypsum gives:
$$ K_{fluorite} = ([Ca^{2+}]_B + x - y)([F^-]-0.5y)^2 $$
so
$$ ([Ca^{2+}]_B + x - y)- K_{fluorite} /([F^-]-0.5y)^2 = 0 $$

In a similar fashion we can write for SO4-2:
$$ ([SO_4^{-2}]_B + x)- K_{gypsum} /([Ca^{2+}] + x - y) = 0 $$


In [5]:
def my_fun(par):
    x = par[0]
    y = par[1]
    
    # we first estimate Ca_C from the fluorite product
    Ca_C = max([Ca_B + x - y,1e-20])
    F_C = max([F_B - 0.5*y,1e-20])
    SO4_C = max([SO4_B + x,1e-20])
    
    # then we estimate SO4_C from the gypsum product and Ca_C
    
    res0 = np.log10(Ca_C) + 2*np.log10(F_C) - logKeqFluorite
    res1 = np.log10(Ca_C) + np.log10(SO4_C) - logKeqGyps
    ssr = res0**2 + res1**2
    
    return ssr

In [6]:
sol = scipyo.minimize(my_fun, [0.005,0.005], method = 'Nelder-Mead')
print(sol)



       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 0.1583595044579776
             x: [ 8.454e-03  8.824e-04]
           nit: 32
          nfev: 61
 final_simplex: (array([[ 8.454e-03,  8.824e-04],
                       [ 8.455e-03,  8.819e-04],
                       [ 8.452e-03,  8.841e-04]]), array([ 1.584e-01,  1.584e-01,  1.584e-01]))


In [7]:
x = sol.x[0]
y = sol.x[1]
Ca_C = Ca_B + x - y 
F_C = F_B - 0.5*y
SO4_C = SO4_B + x

print('Ca+2 : ' + str(Ca_C))
print('F- : ' + str(F_C))
print('SO4-2 : ' + str(SO4_C))


Ca+2 : 0.007679499068636391
F- : 5.8808864630983276e-05
SO4-2 : 0.008561881339374425
